In [1]:
import pandas as pd
import numpy as np
import string
import random
import pickle
from tqdm import tqdm
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
from gensim.models.doc2vec import TaggedDocument
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pyresparser import ResumeParser
import re

In [15]:
###DECLARE CV PATH###
path = '/home/ftraverso/code/francescotraverso/job_predictor/data/cv_directory/CV_Example.pdf'

###OPEN RESUME###
resume = ResumeParser(path).get_extracted_data()

###EXTRACT JOB EXPERIENCE###
JD = " ".join(resume['experience'])
JD

/home/ftraverso/.pyenv/versions/3.8.12/envs/job_predictor-env/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


'Management Consulting in FREESCG - Food Consultant - Management consultancy specialized in food & beverage - Consulting services for restaurants, bars and similar - Project developed with specialization to each client through a diagnosis and an action plan put into practice, with analysis of the obtained results - Areas of expertise: Financial, Logistics, Human Resources, Marketing, Administrative Backoffice, Production and Operation Analysis and Management of Purchasing and Logistics in MADPIZZA - Evaluation, restructuring and monitoring of existing logistics department - Establish two-way communication procedures with an emphasis on motivation and teamwork - Consumption forecast - Supplier assessment, prospecting and monitoring - Settle divergences or create quick solutions - Creation of new work tools - Supply chain assessment and identification of improvement points - Supervision of stock and orders of each cost center in order to reduce inventory costs - Order Validation by Point

In [3]:
############################
### describe your job!!! ###
############################

describe_your_job = JD + DS_skills

##################################
### get 'n' job suggeestion!!! ###
##################################

n_jobs = 10


In [4]:
# get dataframes from CSV files

df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_skills.csv')

/tmp/ipykernel_29694/608757610.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_skills.csv')


In [5]:
# filter unneeded columns of of the dataframe and add needed ones

df_occ_n_skills = df_occ_n_skills.filter(items=['preferredLabel', 'description', 'skills'])
df_occ_n_skills = df_occ_n_skills.reindex(columns=['preferredLabel','description', 'skills'])
df_occ_n_skills.rename(columns={'preferredLabel': 'job_title'}, inplace=True)
df_occ_n_skills['description_input'] = 0
df_occ_n_skills['skills_input'] = 0

In [6]:
# create description_input and skills_input, which are strings on which the model will be fit

for row, index in tqdm(df_occ_n_skills.iterrows()):
    underscored_job_title = index['job_title'].replace(" ", "_")
    this_rows_description_input = underscored_job_title + ' ' + index['description']
    this_rows_skills_input = underscored_job_title + ' ' + index['skills']
    df_occ_n_skills.iloc[row,-2] = this_rows_description_input
    df_occ_n_skills.iloc[row,-1] = this_rows_skills_input

35824it [00:05, 7091.98it/s]


In [7]:
# inspect dataframe

df_occ_n_skills.head()

,job_title,description,skills,description_input,skills_input
0,technical director,Technical directors realise the artistic visio...,"adapt to artists' creative demands, organise r...",technical_director Technical directors realise...,technical_director adapt to artists' creative ...
1,metal drawing machine operator,Metal drawing machine operators set up and ope...,"cold drawing processes, monitor moving workpie...",metal_drawing_machine_operator Metal drawing m...,metal_drawing_machine_operator cold drawing pr...
2,precision device inspector,Precision device inspectors make sure precisio...,"precision measuring instruments, monitor machi...",precision_device_inspector Precision device in...,precision_device_inspector precision measuring...
3,air traffic safety technician,Air traffic safety technicians provide technic...,"air transport law, aircraft flight control sys...",air_traffic_safety_technician Air traffic safe...,air_traffic_safety_technician air transport la...
4,hospitality revenue manager,Hospitality revenue managers maximise revenue ...,"develop revenue generation strategies, ensure ...",hospitality_revenue_manager Hospitality revenu...,hospitality_revenue_manager develop revenue ge...


In [8]:
# get entire dataframe as dataset according to Qiewi's suggestion
# concatenate the job_title:skills list to the end of df_occ_n_skills

X_all = pd.concat([df_occ_n_skills['description_input'], df_occ_n_skills['skills_input']]).reset_index(drop=True)

In [9]:
# applying preprocessing to corpus as in Gensim tutorial, also applying Phraser

def read_corpus(corpus):
   
    # instantiate Phraser outside of the loop
    sentence_stream = [entry.split(" ") for entry in corpus]
    bigrams = Phrases(
        sentence_stream,
        min_count=5,
        threshold=5,
        connector_words=ENGLISH_CONNECTOR_WORDS
        )
    
    for i, line in enumerate(corpus):    
    
        # remove punctuation
        for punctuation in string.punctuation:        
            sentence = line.replace(punctuation, '')

        # remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = word_tokenize(sentence)
        stopword_free_tokens = [token for token in tokens if token not in stop_words]
        sentence = ' '.join(stopword_free_tokens)

        # lemmatize
        sentence = WordNetLemmatizer().lemmatize(sentence, pos='n')
        sentence = WordNetLemmatizer().lemmatize(sentence, pos='v')
        
        # get bigrams
        sent = sentence.split()

        # yield tagged final corpus
        yield TaggedDocument(bigrams[sent], [i])

all_corpus = list(read_corpus(X_all))

In [10]:
# Preprocessing function for job descriptions

def preprocess_input(sentence):
    
    # remove punctuation
    for punctuation in string.punctuation:        
        sentence = sentence.replace(punctuation, '')
    
    # set lowercase
    sentence = sentence.lower()
    
    # remove numbers
    sentence = ''.join(char for char in sentence if not char.isdigit())
    
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(sentence)
    stopword_free_tokens = [token for token in tokens if token not in stop_words]
    sentence = ' '.join(stopword_free_tokens)

    # lemmatize
    sentence = WordNetLemmatizer().lemmatize(sentence, pos='n')
    sentence = WordNetLemmatizer().lemmatize(sentence, pos='v')
    
    # split into tokens again after Lemmatizing --- this was replaced by Phraser 
    # sentence = word_tokenize(sentence)
    
    return sentence

new_description = preprocess_input(describe_your_job)

In [11]:
new_description

'technologies industries diebold nixdorf sigma bs lastly associated al mutaafi trading khartoum sudan atm department manager reported director successfully managed entire atm department including department setup finding good brand atm certification central bank sudan team hiring development train engineers recognized top management istinara matc khartoum sudan increasing number customers customers reach approximately market share atm sudan selling diebold nixdorf atms units atmeye establishing atm operations department increase uptime percentage machines alfa trading installing configuring atms belong banks sudan migrate atms old national host sm new national host campus upgrade atm software ndx ndx xv first implementation atmeye cctv system sudan support company erp system talent hiringteam training development organi zati onal experi enc e may ’ march al mutaafi trading khartoum–sudan atm department manager lastly managed entire functions business operations al mutaafi trading inclu

In [12]:
# load saved BERT model and its embedding from disk

filename = '../model/bert_model.sav'
bert_model = pickle.load(open(filename, 'rb'))

filename = '../model/all_corpus_embed.sav'
all_corpus_embed = pickle.load(open(filename, 'rb'))

In [13]:
# run model

# run input through the model
new_description_embed = bert_model.encode(new_description)

# calculate and order cosine similarity
similarity_rank = cosine_similarity([new_description_embed], all_corpus_embed)
similarity_rank_index = np.argsort(similarity_rank[0])[::-1]

In [14]:
print(f'TEST DOCUMENT: {new_description} \n')
print('LISTING 10 MOST SIMILAR JOB ROLES & DESCRIPTIONS \n')

for i in range(n_jobs):
    
    if similarity_rank_index[i] <= len(df_occ_n_skills):
        new_index = similarity_rank_index[i]
    else:
        new_index = similarity_rank_index[i] - len(df_occ_n_skills)

    print(f'RANK #{i+1}: ' + df_occ_n_skills.loc[new_index]['job_title'])
    print(df_occ_n_skills.loc[new_index]['description'])
    print(f'Similarity score: {round(similarity_rank[0][similarity_rank_index[i]]*100,1)} %')
    print(f'Index in dataframe: {new_index} \n')



TEST DOCUMENT: technologies industries diebold nixdorf sigma bs lastly associated al mutaafi trading khartoum sudan atm department manager reported director successfully managed entire atm department including department setup finding good brand atm certification central bank sudan team hiring development train engineers recognized top management istinara matc khartoum sudan increasing number customers customers reach approximately market share atm sudan selling diebold nixdorf atms units atmeye establishing atm operations department increase uptime percentage machines alfa trading installing configuring atms belong banks sudan migrate atms old national host sm new national host campus upgrade atm software ndx ndx xv first implementation atmeye cctv system sudan support company erp system talent hiringteam training development organi zati onal experi enc e may ’ march al mutaafi trading khartoum–sudan atm department manager lastly managed entire functions business operations al mutaafi